In [1]:
import numpy as np
import pandas as pd
import pickle
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
!pip install datasets

In [3]:
import nltk

In [4]:
data =pd.DataFrame()

In [5]:
with open("lemmatized_tweets (1).pkl","rb") as file_handle:
    data1 = pickle.load(file_handle)

In [6]:
data['text'] = data1

In [7]:
data['text']

0          [twitpic, com, zl, awww, bummer, shoulda, get,...
1          [upset, update, facebook, texte, cry, result, ...
2                                [manage, save, rest, bound]
3                            [body, feel, itchy, like, fire]
4                                              [behave, mad]
                                 ...                        
1599995                           [wake, school, good, feel]
1599996      [thewdb, com, cool, hear, old, walt, interview]
1599997                 [ready, mojo, makeover, ask, detail]
1599998    [happy, th, birthday, boo, alll, time, tupac, ...
1599999    [happy, charitytuesday, thenspcc, sparkscharit...
Name: text, Length: 1600000, dtype: object

In [8]:
converted_raw_text = list(data["text"].apply(lambda x: " ".join(x)))

In [9]:
converted_raw_text = list(filter(lambda x: len(x) > 1,converted_raw_text))

In [10]:
first_100000_tweets = converted_raw_text[:100000]

# Save to a text file
with open('first_100000_tweets.txt', 'w', encoding='utf-8') as file:
    for tweet in first_100000_tweets:
        file.write(tweet + '\n')

print("First 100000 tweets have been saved to 'first_100000_tweets.txt'.")

First 100000 tweets have been saved to 'first_100000_tweets.txt'.


In [11]:
import datasets
dset = datasets.load_dataset('text', data_files={"train":'first_100000_tweets.txt'})

/home/vivek123/project/work2vec/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100000 examples [00:00, 1818550.12 examples/s]


In [12]:
def tokenize_tweets(single_row):
    single_row['tokenized-tweets'] = single_row["text"].split()
    return single_row

In [13]:
dset = dset.map(tokenize_tweets)

Map: 100%|██████████| 100000/100000 [00:02<00:00, 34872.54 examples/s]


In [14]:
dset['train']['tokenized-tweets']

[['twitpic',
  'com',
  'zl',
  'awww',
  'bummer',
  'shoulda',
  'get',
  'david',
  'carr',
  'day'],
 ['upset',
  'update',
  'facebook',
  'texte',
  'cry',
  'result',
  'school',
  'today',
  'blah'],
 ['manage', 'save', 'rest', 'bound'],
 ['body', 'feel', 'itchy', 'like', 'fire'],
 ['behave', 'mad'],
 ['need', 'hug'],
 ['yes', 'rain', 'bit', 'bit', 'lol', 'fine', 'thank'],
 ['nope'],
 ['spring', 'break', 'plain', 'city', 'snowing'],
 ['pierce', 'ear'],
 ['think', 'ua', 'loss', 'embarrass'],
 ['know', 'talk', 'anymore'],
 ['gun', 'zac', 'snyder', 'doucheclown'],
 ['miss', 'iamlilnicki', 'premiere'],
 ['hollis',
  'death',
  'scene',
  'hurt',
  'severely',
  'watch',
  'film',
  'wry',
  'director',
  'cut'],
 ['file', 'taxis'],
 ['drink', 'forget', 'table', 'drink'],
 ['friend', 'call', 'asked', 'meet', 'mid', 'valley', 'today', 'time', 'sigh'],
 ['barista', 'bake', 'cake', 'ate'],
 ['week', 'go', 'hope'],
 ['blagh', 'class', 'tomorrow'],
 ['hate', 'wake', 'people'],
 ['go', 'c

In [15]:
!pip install ngrams

In [16]:
from nltk.util import ngrams

def convert_to_trigrams(single_row):
    single_row["tri-grams"] = list(ngrams(single_row['tokenized-tweets'],n=3))
    return single_row

In [17]:
dset['train'] = dset['train'].map(convert_to_trigrams)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map: 100%|██████████| 100000/100000 [00:05<00:00, 18463.47 examples/s]


In [18]:
dset

DatasetDict({
    train: Dataset({
        features: ['text', 'tokenized-tweets', 'tri-grams'],
        num_rows: 100000
    })
})

In [19]:
converted_raw_text[0]

'twitpic com zl awww bummer shoulda get david carr day'

In [20]:
vocabulary = set()
for raw_text in converted_raw_text[:100000]:
    vocabulary.update(raw_text.split())

In [21]:
len(vocabulary)

43953

In [22]:
vocabulary

{'thi',
 'convntion',
 'crane',
 'huaaah',
 'shizah',
 'fyngalno',
 'sundaymorning',
 'annaresa',
 'aly',
 'ponderland',
 'fraudster',
 'titan',
 'beak',
 'otu',
 'misrepresent',
 'bocha',
 'kum',
 'nooooooooooooooo',
 'wedding',
 'dirkie',
 'anyy',
 'whoopie',
 'thristy',
 'sardines',
 'vmware',
 'krew',
 'lookout',
 'hours',
 'mufo',
 'stony',
 'bambooooozle',
 'goonyttt',
 'borin',
 'daay',
 'case',
 'quietly',
 'boarder',
 'rooftop',
 'nocturnal',
 'flyer',
 'jamilla',
 'stoie',
 'quackkkkk',
 'ijuswannasleep',
 'solar',
 'phase',
 'romans',
 'uplode',
 'pittsburgh',
 'bellamuerte',
 'pmssse',
 'truee',
 'territory',
 'wedge',
 'nouvelle',
 'hernia',
 'texted',
 'vividly',
 'past',
 'cowell',
 'twistori',
 'gonsenheim',
 'awkward',
 'nestor',
 'looky',
 'whyohwhyohwhy',
 'munchkinster',
 'illinoise',
 'knowledgable',
 'bobbieb',
 'nneka',
 'usps',
 'viguera',
 'editione',
 'wgr',
 'angga',
 'ginger',
 'ja',
 'apx',
 'package',
 'maternity',
 'revison',
 'daugh',
 'textiles',
 'sunb

In [23]:
vocab2idx = dict(zip(vocabulary, range(len(vocabulary))))

In [24]:
vocab2idx

{'thi': 0,
 'convntion': 1,
 'crane': 2,
 'huaaah': 3,
 'shizah': 4,
 'fyngalno': 5,
 'sundaymorning': 6,
 'annaresa': 7,
 'aly': 8,
 'ponderland': 9,
 'fraudster': 10,
 'titan': 11,
 'beak': 12,
 'otu': 13,
 'misrepresent': 14,
 'bocha': 15,
 'kum': 16,
 'nooooooooooooooo': 17,
 'wedding': 18,
 'dirkie': 19,
 'anyy': 20,
 'whoopie': 21,
 'thristy': 22,
 'sardines': 23,
 'vmware': 24,
 'krew': 25,
 'lookout': 26,
 'hours': 27,
 'mufo': 28,
 'stony': 29,
 'bambooooozle': 30,
 'goonyttt': 31,
 'borin': 32,
 'daay': 33,
 'case': 34,
 'quietly': 35,
 'boarder': 36,
 'rooftop': 37,
 'nocturnal': 38,
 'flyer': 39,
 'jamilla': 40,
 'stoie': 41,
 'quackkkkk': 42,
 'ijuswannasleep': 43,
 'solar': 44,
 'phase': 45,
 'romans': 46,
 'uplode': 47,
 'pittsburgh': 48,
 'bellamuerte': 49,
 'pmssse': 50,
 'truee': 51,
 'territory': 52,
 'wedge': 53,
 'nouvelle': 54,
 'hernia': 55,
 'texted': 56,
 'vividly': 57,
 'past': 58,
 'cowell': 59,
 'twistori': 60,
 'gonsenheim': 61,
 'awkward': 62,
 'nestor': 6

In [25]:
len(vocab2idx)

43953

In [26]:
def convert_to_bigrams(single_row):
    center_token_token_pairs = list()
    for single_trigrams in single_row["tri-grams"]:
        bigrams = list()
        bigrams.append([vocab2idx[single_trigrams[1]], vocab2idx[single_trigrams[0]]])
        bigrams.append([vocab2idx[single_trigrams[1]], vocab2idx[single_trigrams[2]]])
        center_token_token_pairs.append(bigrams)
    single_row['tri-grams'] = center_token_token_pairs
    
    return single_row

In [27]:
dset['train'] = dset['train'].map(convert_to_bigrams)

Map: 100%|██████████| 100000/100000 [00:09<00:00, 10819.63 examples/s]


In [28]:
input_token_target_token_pairs = list()
for single_tweet_bigrams in dset['train']['tri-grams']:
    for bigrams_list in single_tweet_bigrams:
        input_token_target_token_pairs.append(bigrams_list[0])
        input_token_target_token_pairs.append(bigrams_list[1])
        

In [29]:
input_token_target_token_pairs[:10]

[[42677, 5078],
 [42677, 38039],
 [38039, 42677],
 [38039, 34074],
 [34074, 38039],
 [34074, 26464],
 [26464, 34074],
 [26464, 25030],
 [25030, 26464],
 [25030, 22921]]

In [30]:
len(input_token_target_token_pairs)

795998

In [31]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [32]:
import torch
class SkipGramDataset(torch.utils.data.Dataset):
    def __init__(self, input_target_pairs):
        self.data = input_target_pairs
        
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)


In [33]:
training_data_obj = SkipGramDataset(input_token_target_token_pairs)

In [34]:
training_data_generator = torch.utils.data.DataLoader(training_data_obj, batch_size=32,
                                                      num_workers= os.cpu_count())

In [35]:
for mini_batch in training_data_generator:
    print(mini_batch)
    break

[tensor([42677, 42677, 38039, 38039, 34074, 34074, 26464, 26464, 25030, 25030,
        22921, 22921,  1634,  1634,  6827,  6827,  6781,  6781,   888,   888,
         4264,  4264, 14073, 14073, 22854, 22854,  2437,  2437,  3436,  3436,
         6886,  6886]), tensor([ 5078, 38039, 42677, 34074, 38039, 26464, 34074, 25030, 26464, 22921,
        25030,  1634, 22921,  6827,  1634,  5768, 27266,   888,  6781,  4264,
          888, 14073,  4264, 22854, 14073,  2437, 22854,  3436,  2437, 12198,
        15463, 22667])]


In [36]:
class Word2VecSkipGramNeuralNetwork(nn.Module):
    
    def __init__(self,vocabulary_size, topic_vector_dim):
        super().__init__()
        
        self.hidden_layer = torch.nn.Embedding(num_embeddings=vocabulary_size,
                                               embedding_dim=topic_vector_dim)
        
        self.output_layer  = torch.nn.Linear(in_features=topic_vector_dim
                                             , out_features=vocabulary_size)
        
        self.output_layer_activation = torch.nn.Softmax()
        def forward(self, input_word_indices):
            return self.hidden_layer(input_word_indices)
        
        
        
    def forward(self, center_token):
        
        embedding_layer_out  =  self.hidden_layer(center_token)
        linear_layer_out = self.output_layer(embedding_layer_out)
        nn_out = self.output_layer_activation(linear_layer_out)
        return nn_out

    

In [61]:
our_word2v_skipgram_nw= Word2VecSkipGramNeuralNetwork(len(vocab2idx), 64)

In [56]:
for mini_batch_idx, mini_batch in enumerate(training_data_generator):
    print("index{}".format(mini_batch_idx))
    print("center token{}".format(mini_batch[0]))
    print("index{}".format(mini_batch[1]))
    break


index0
center tokentensor([42677, 42677, 38039, 38039, 34074, 34074, 26464, 26464, 25030, 25030,
        22921, 22921,  1634,  1634,  6827,  6827,  6781,  6781,   888,   888,
         4264,  4264, 14073, 14073, 22854, 22854,  2437,  2437,  3436,  3436,
         6886,  6886])
indextensor([ 5078, 38039, 42677, 34074, 38039, 26464, 34074, 25030, 26464, 22921,
        25030,  1634, 22921,  6827,  1634,  5768, 27266,   888,  6781,  4264,
          888, 14073,  4264, 22854, 14073,  2437, 22854,  3436,  2437, 12198,
        15463, 22667])


In [74]:
from tqdm import tqdm

epochs = 5 
optimizer = torch.optim.SGD(params = our_word2v_skipgram_nw.parameters(),
                             lr = 0.5)
loss_fn = torch.nn.CrossEntropyLoss()
progress_bars = tqdm(range(epochs*len(training_data_generator)))

for epoch in range(epochs):
    for mini_batch_idx, mini_batch in enumerate(training_data_generator):
        
        center_token_mini_batch = mini_batch[0]
        surrounding_token_mini_batch = mini_batch[1]
        
        center_token_mini_batch.to('cpu')
        surrounding_token_mini_batch.to('cpu')
        
        optimizer.zero_grad()
        
        if(mini_batch_idx+1) %500==0:
            print("epoch #{}, Time step# {}, Loss= {}".format(epoch,(mini_batch_idx+1), loss_fn_value))
            
        y_pred = our_word2v_skipgram_nw(center_token_mini_batch)
        loss_fn_value = loss_fn(y_pred, surrounding_token_mini_batch)
        loss_fn_value.backward()
        
        optimizer.step()
        progress_bars.update(1)

  0%|          | 505/124375 [00:13<55:17, 37.34it/s]  

epoch #0, Time step# 500, Loss= 10.690958976745605


  1%|          | 1004/124375 [00:28<1:05:23, 31.45it/s]

epoch #0, Time step# 1000, Loss= 10.659703254699707


  1%|          | 1503/124375 [00:44<57:55, 35.35it/s]  

...

epoch #0, Time step# 3000, Loss= 10.66030216217041


  3%|▎         | 3503/124375 [01:49<1:02:21, 32.30it/s]

epoch #0, Time step# 3500, Loss= 10.690949440002441


  3%|▎         | 3724/124375 [01:56<1:01:27, 32.72it/s]

KeyboardInterrupt: 